In [1]:
###Import Block
import pandas as pd
import numpy as np
import math
import cv2
import tensorflow as tf
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics as mt
from sklearn.metrics import roc_curve
from skimage.io import imshow
from matplotlib import pyplot as plt
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from sklearn.metrics import auc
import seaborn as sns
from keras.callbacks import EarlyStopping
from keras.regularizers import l2 

#print entire array
import sys
np.set_printoptions(threshold=sys.maxsize)

In [2]:
# Load trained model
cnn = keras.models.load_model("cnn_model")
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 128)       7

In [3]:
cv2.ocl.setUseOpenCL(False)

labels = {0: "Happy", 1: "Sad", 2: "Surprised"}

# start the webcam
cam = cv2.VideoCapture(0)
while True:

    is_capturing, frame = cam.read()
    if not is_capturing:
        break
    facecasc = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray,1.1,4)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
        
        reshapeImg = cropped_img.reshape((1,48,48,1))
        normalizedImg = (reshapeImg - np.amin(reshapeImg)) / (np.amax(reshapeImg) - np.amin(reshapeImg))
        normalizedImg = 2*normalizedImg-1 #s
        
        yhat_cnn = np.argmax(cnn.predict(normalizedImg), axis=1)

        cv2.putText(frame, labels[yhat_cnn[0]], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(1600,960),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('a'):
        break

cam.release()
cv2.destroyAllWindows()